# The Ant System Algorithm For The Travelling Salesman Problem

Algorithm from the Book **Ant Colony Optimization by Marco Dorigo and Thomas Stützle**

In [2]:
#----------------------------------------
# Chia E. Tungom
# chemago99@yahoo.com
# July 30th 2020
# Shenzhen China
#----------------------------------------

In [3]:
# import libraries

import numpy as np
import math
import random

# Roulette Wheel for Probability

In [4]:
# probability and selection using roulette wheel 

# Calculate cumulative sum for roulette wheel 
def PPi(pi):
    n = len(pi)
    ans = []
    
    for i in range(n):
        ans.append( sum([i for i in pi[:i+1]]) )
    return ans 

# Roulette Wheel 
def Roulette(ppi, Pop):
    n = len(ppi)
    ra = random.random()
    
    for i in range(n):
        if i == 0 :
            if ra > 0 and ra < ppi[i]:
                return Pop[i]
        else:
            if ra > ppi[i-1] and ra < ppi[i]:
                return Pop[i]

# Calculate Route Cost

In [5]:
# calculate cost of each route
def Cost(Pop, Cmatrix):
    Cvector = []
    
    for i in range(len(Pop)):
        route = Pop[i]
        cost = 0
        
        for j in range(len(route)-1):
            cost += Cmatrix[route[j]][route[j+1]]
        cost += Cmatrix[route[0]][route[len(route)-1]]   
        Cvector.append(cost)
        
    return Cvector

# Initialize Pheromone 

In [6]:

def TSPher(graph):
    
    n = len(graph)
    route = [i for i in range(n)]
    random.shuffle(route)
    
    cost = Cost([route], graph)[0]
    pher = [[cost for i in range(n)] for j in range(n)]
    
    return pher

# Calculate Transition Probability

In [48]:
# Transition Probability
def TransitionP(connections, eps):
    """ Takes a list of tuples [(pheromone, distance, hob), ... , (pheromone, distance, hob)]
        return trasition probability for each hob [(probability, hob), ... , (probability, hob)]"""
    
    total = sum( [ (i[0]*eps)*(i[1]*(1-eps)) for i in connections])    # sum heuristic and pheromone info
    prob = [( ((i*eps)*(j*(1-eps))/total) , k) for i,j,k in connections ]  # probability of each hob
    
    return prob

# pheromone, distance and hob information
def DPH(phermatrix, graph, connections, preshob):
    """ DPH(Distance Pheromone hob)
        takes a pheromone matrix
        graph matrix 
        connections in graph to use 
        present hob in graph
        and return the pheromone, distance and hob [(pheromone, distance, hob), ... , (pheromone, distance, hob)]"""
    
    dph = []
    for i in range(len(connections)):
        if connections[i] > 0:
            dph.append( (phermatrix[preshob][i], 1/graph[preshob][i], i) )
            
    #dph = [(phermatrix[preshob][i], graph[preshob][i], i) for i in range(len(connections)) if connections[i] > 0 ]
    
    return dph

def Disconnect(visited, connections):
    """ Disconnects already visited hobs
    visited: already visited hobs
    connections: connection vector with 0: no connection x: length of connection"""
    
    for i in visited:
        connections[i] = 0
        #print(i, visited, connections)
    #print("DONE++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++DONE")
    return connections

# Find Path 

In [49]:
def FindPath(phermatrix, graph, takeoff, eps = 0.5):
    
    path = []
    path.append(takeoff)
    
    while len(path) != len(graph):
        
        Cons = graph[takeoff].copy()
        tabu = path.copy()
        Cons = Disconnect(tabu, Cons)
        HP = DPH(phermatrix, graph, Cons, takeoff)
        prob = TransitionP(HP, eps)
        takeoff = Roulette(PPi([i[0] for i in prob]), [i[1] for i in prob])
        
        path.append(takeoff)
 
    return path

# Update Pheromone 

In [112]:
# Update Pheromone in visited arcs

def UpPheromone(route, matrix):
    """route : route taken
    matrix: pheromone matrix"""
    
    tau = 1/Cost([route], matrix)[0]
    #tau = 1
    
    for i in range(len(route)-1):
        
        matrix[route[i]][route[i+1]] += tau
        matrix[route[i+1]][route[i]] += tau
    
    # add pheromone to last and first route for TSP
    matrix[route[-1]][route[0]] += tau
    matrix[route[0]][route[-1]] += tau

    return matrix


# Ant Cycle
def UpPheromoneANTC(route, matrix, Q = 1):
    """route : route taken
    matrix: pheromone matrix
    Q: a constant """
    
    tau = Q/Cost([route], matrix)[0]
    #tau = 1
    
    for i in range(len(route)-1):
        
        matrix[route[i]][route[i+1]] += tau
        matrix[route[i+1]][route[i]] += tau
    
    # add pheromone to last and first route for TSP
    matrix[route[-1]][route[0]] += tau
    matrix[route[0]][route[-1]] += tau

    return matrix

# Ant Density
def UpPheromoneANTD(route, matrix, Q = 1):
    """route : route taken
    matrix: pheromone matrix
    Q: a constant """
   
    for i in range(len(route)-1):
        
        matrix[route[i]][route[i+1]] += Q
        matrix[route[i+1]][route[i]] += Q
    
    # add pheromone to last and first route for TSP
    matrix[route[-1]][route[0]] += Q
    matrix[route[0]][route[-1]] += Q

    return matrix

# Ant Quantity
def UpPheromoneANTQ(route, matrix, Q = 1):
    """route : route taken
    matrix: pheromone matrix
    Q: a constant """
   
    for i in range(len(route)-1):
        
        tau = Q/matrix[route[i]][route[i+1]]
        matrix[route[i]][route[i+1]] += tau
        matrix[route[i+1]][route[i]] += tau
    
    # add pheromone to last and first route for TSP
    tau = Q/matrix[route[-1]][route[0]]
    matrix[route[-1]][route[0]] += tau
    matrix[route[0]][route[-1]] += tau

    return matrix



# ANTS
def UpPheromoneANTS(route, matrix, Lavg, LB, V = 0.1):
    """route : route taken
    matrix: pheromone matrix
    Q: a constant """
    
    numer = Cost([route], matrix)[0] - LB
    den = Lavg - LB + 0.001
    
    tau = V*( 1 - (numer/den))
    #tau = 1
    
    for i in range(len(route)-1):
        
        matrix[route[i]][route[i+1]] += tau
        matrix[route[i+1]][route[i]] += tau
    
    # add pheromone to last and first route for TSP
    matrix[route[-1]][route[0]] += tau
    matrix[route[0]][route[-1]] += tau

    return matrix

# Evaporate Pheromone 

In [113]:
def EvPheromone(matrix, rho):
    
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            if matrix[i][j] > 0:
                matrix[i][j] = (1-rho)*matrix[i][j]
            else:
                matrix[i][j] = math.exp(-5)
    return matrix

def EvPheromoneNeg(matrix, rho):
    
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            matrix[i][j] = (1-rho)*matrix[i][j]
        
    return matrix

# Generate a Toy Symmetric TSP Problem 

In [114]:
# A random graph
def Graph(dim, bounds):
    b = [(i, str(i)) for i in range(dim)]   
    matrix = []
    
    for i,j in b:
        j=[]
        if i == 0:
            j.append(0)
            matrix.append(j)
        else:
            for k in range(i+1):
                if k == i:
                    j.append(0)
                else:
                    j.append(random.choice([random.randint(bounds[0], bounds[1])]))  # route length
            matrix.append(j)
            
    M = matrix        
    for i in range(len(M)):
        
        for j in range(1,len(M)-len(M[i])+1):
            M[i].append(M[i+j][i]) 
            
    return M

# Ant System Algorithm

In [120]:
def AS(Ngraph, ants, eps = 0.5, rho = 0.5, runs = 50, Pher = None):
    
    if Pher == None:
        Pher = TSPher(Ngraph) 
        
    starthob = random.choice([i for i in range(len(Ngraph))])
    i = 0
    while i < runs:
        Ants = []
        #starthob = random.choice([i for i in range(len(Ngraph))])
        
        for j in range(ants):
            stof = FindPath(Pher, Ngraph, starthob, eps)
            Ants.append(stof)
        
        # get Lavg and LB which are needed for Pheromone Update
        TCost = Cost(Ants, Ngraph)
        Lavg, LB = sum(TCost)/len(Ants), min(TCost) 
   
        #Pheromone update
        for j in Ants:
            Pher = UpPheromone(j, Pher)
            #UpPheromoneANTS(j, Pher, Lavg, LB, V = 1)  # ant cycle
            #UpPheromoneANTD(j, Pher, 1)  # ant density
            #UpPheromoneANTQ(j, Pher, 1)  # Ant Quantity
            
        #Pheromone update  
        #EvPheromone(Pher, rho)
        EvPheromoneNeg(Pher, rho)
        
        if i%int(runs*0.05) == 0:
            C = Cost(Ants, Ngraph)
            print("In Generation ", i, " The Minimum Cost Found is = ", min(C), "  and the Maximum Cost Found is = ", max(C))
            
        i += 1 

    C = Cost(Ants, Ngraph)    
    print("Minimum Cost Found = ", min(C), "Maximum Cost Found = ", max(C))
    return np.array(Ants), min(C) #, np.array(Pher)

# Example on the Toy Problem

In [121]:
# Toy Network graph 
D = 10
bounds = (2,10)
random.seed(10)
graph = Graph(D, bounds)

np.array(graph)

array([[ 0,  2,  9,  9,  7,  8,  5, 10,  6,  9],
       [ 2,  0,  5,  4,  5,  6,  8,  8, 10,  6],
       [ 9,  5,  0, 10,  2,  9,  2,  3,  4, 10],
       [ 9,  4, 10,  0,  4,  6,  4, 10,  7,  3],
       [ 7,  5,  2,  4,  0,  4,  6,  5,  3,  9],
       [ 8,  6,  9,  6,  4,  0,  5,  5,  9,  4],
       [ 5,  8,  2,  4,  6,  5,  0,  2,  5,  4],
       [10,  8,  3, 10,  5,  5,  2,  0,  8,  7],
       [ 6, 10,  4,  7,  3,  9,  5,  8,  0,  8],
       [ 9,  6, 10,  3,  9,  4,  4,  7,  8,  0]])

In [122]:
# Find Shortest Path with the Algorithm

ants = len(graph)
eps = 0.5
rho = 0.5
runs = 200

AS(graph, ants, eps, rho, runs)

In Generation  0  The Minimum Cost Found is =  48   and the Maximum Cost Found is =  61
In Generation  10  The Minimum Cost Found is =  34   and the Maximum Cost Found is =  56
In Generation  20  The Minimum Cost Found is =  34   and the Maximum Cost Found is =  60
In Generation  30  The Minimum Cost Found is =  33   and the Maximum Cost Found is =  46
In Generation  40  The Minimum Cost Found is =  34   and the Maximum Cost Found is =  51
In Generation  50  The Minimum Cost Found is =  33   and the Maximum Cost Found is =  57
In Generation  60  The Minimum Cost Found is =  33   and the Maximum Cost Found is =  38
In Generation  70  The Minimum Cost Found is =  33   and the Maximum Cost Found is =  53
In Generation  80  The Minimum Cost Found is =  33   and the Maximum Cost Found is =  46
In Generation  90  The Minimum Cost Found is =  33   and the Maximum Cost Found is =  58
In Generation  100  The Minimum Cost Found is =  37   and the Maximum Cost Found is =  57
In Generation  110  T

(array([[6, 9, 3, 1, 0, 8, 2, 4, 5, 7],
        [6, 7, 3, 1, 0, 8, 4, 2, 9, 5],
        [6, 2, 7, 5, 9, 3, 4, 8, 0, 1],
        [6, 7, 8, 2, 4, 3, 1, 0, 9, 5],
        [6, 7, 2, 4, 3, 9, 5, 1, 0, 8],
        [6, 2, 7, 5, 1, 0, 8, 4, 3, 9],
        [6, 2, 4, 8, 7, 5, 1, 0, 9, 3],
        [6, 3, 9, 8, 0, 1, 5, 7, 2, 4],
        [6, 2, 4, 3, 1, 0, 8, 9, 5, 7],
        [6, 7, 5, 9, 3, 4, 2, 8, 0, 1]]), 36)

# Soving Real TSP problems 

In [123]:
# att48

import RouteMatrix as RM
path = './Data/att48.txt'
graph = RM.TSRM(path)


ants = len(graph)
eps = 0.5
rho = 0.5
runs = 200

AS(graph, ants, eps, rho, runs)

In Generation  0  The Minimum Cost Found is =  83655.31673029058   and the Maximum Cost Found is =  143681.68562239333
In Generation  10  The Minimum Cost Found is =  91043.505984267   and the Maximum Cost Found is =  129572.73498254821
In Generation  20  The Minimum Cost Found is =  64339.97557763282   and the Maximum Cost Found is =  111595.28336183216
In Generation  30  The Minimum Cost Found is =  45123.56090683419   and the Maximum Cost Found is =  64907.87376234885
In Generation  40  The Minimum Cost Found is =  43046.28201670407   and the Maximum Cost Found is =  69379.7140268021
In Generation  50  The Minimum Cost Found is =  42369.94425606348   and the Maximum Cost Found is =  69459.00523668612
In Generation  60  The Minimum Cost Found is =  42005.0284422563   and the Maximum Cost Found is =  66900.73930467971
In Generation  70  The Minimum Cost Found is =  41984.94871264177   and the Maximum Cost Found is =  63822.97112579675
In Generation  80  The Minimum Cost Found is =  43

(array([[13, 12, 20, ..., 32,  6, 17],
        [13, 24, 12, ..., 28, 47, 38],
        [13, 12, 46, ..., 20, 40, 23],
        ...,
        [13, 24, 12, ..., 25,  3, 34],
        [13, 24, 12, ..., 31, 43,  1],
        [13, 24, 12, ..., 33,  2, 21]]), 36535.86051220195)

In [119]:
import RouteMatrix as RM
path = './Data/burma14.txt'
graph = RM.TSRM(path)


ants = len(graph)
#ants = 100
eps = 0.5
rho = 0.5
runs = 1000

AS(graph, ants, eps, rho, runs)

In Generation  0  The Minimum Cost Found is =  43.81822220716433   and the Maximum Cost Found is =  58.45893117647654


TypeError: list indices must be integers or slices, not NoneType